# Manipulating Data in Pandas

We won't cover every possible way to access data in this tutorial, but this should give you a sense of some of the main ways you can access and work with tabular data with Pandas. Things we'll cover:
- Selecting and accessing data from a DataFrame
- Filtering and reindexing data
- Transforming, sorting, aggregating, deduplicating

In [2]:
import pandas as pd
robocall_df = pd.read_csv("Data/Telemarketing_RoboCall_Weekly_Data_Transformed.csv")

We've loaded our data into a DataFrame which is much like a database table, or a single table in a spreadsheet. This table has rows and columns. Pandas has also added an index column which you'll see on the far left of the DataFrame. There is a LOT that a DataFrame can do - you can familiarize yourself with all it offers in the [documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). 

In [3]:
robocall_df

,issues,time_issued,caller_id,phone_number,type_robo,type_telemarketing,state,date_issued,date_created
0,Telemarketing (including do not call and spoof...,NaN,-,-,NaN,NaN,Pennsylvania,NaN,2015-10-01
1,Telemarketing (including do not call and spoof...,2014-07-01 08:20:00,610-990-4243,610-990-4243,NaN,Abandoned Calls,Pennsylvania,2014-07-01,2015-10-01
2,Telemarketing (including do not call and spoof...,2015-09-20 14:00:00,469-656-8497,-,NaN,Prerecorded Voice,Pennsylvania,2015-09-20,2015-10-01
3,Telemarketing (including do not call and spoof...,2015-09-23 11:46:00,717-657-3334,-,NaN,Abandoned Calls,Pennsylvania,2015-09-23,2015-10-01
4,Robocalls,2015-09-28 15:19:00,415-946-5707,-,Prerecorded Voice,NaN,Pennsylvania,2015-09-28,2015-10-01
5,Telemarketing (including do not call and spoof...,2015-09-29 12:00:00,215-624-8359,-,NaN,Live Voice,Pennsylvania,2015-09-29,2015-10-01
6,Telemarketing (including do not call and spoof...,2015-10-01 11:00:00,202-470-3314,-,NaN,Live Voice,Pennsylvania,2015-10-01,2015-10-01
7,Telemarketing (including do not call and spoof...,2015-10-01 10:21:00,567-263-0009,-,NaN,Live Voice,Pennsylvania,2015-10-01,2015-10-01
8,Telemarketing (including do not call and spoof...,2015-10-01 13:02:00,215-763-3788,-,NaN,Prerecorded Voice,Pennsylvania,2015-10-01,2015-10-01
9,Telemarketing (including do not call and spoof...,2015-10-01 12:58:00,717-245-2434,-,NaN,Prerecorded Voice,Pennsylvania,2015-10-01,2015-10-01


If you just want a list of the column names that's easy enough:

In [4]:
robocall_df.columns

Index([u'issues', u'time_issued', u'caller_id', u'phone_number', u'type_robo',
       u'type_telemarketing', u'state', u'date_issued', u'date_created'],
      dtype='object')

To select a single column from the DataFrame you can use the name of the column within brackets:

In [5]:
robocall_df["phone_number"]

0                   -
1        610-990-4243
2                   -
3                   -
4                   -
5                   -
6                   -
7                   -
8                   -
9                   -
10                  -
11                  -
12                  -
13       321-608-4860
14                  -
15                  -
16                  -
17       847-234-9229
18                  -
19                  -
20                  -
21                  -
22                  -
23                  -
24       713-714-5831
25                  -
26       876-254-1483
27                  -
28       240-345-1622
29       786-275-5986
             ...     
38955               -
38956               -
38957               -
38958    201-380-0014
38959    815-264-2191
38960    800-203-0028
38961    312-637-5631
38962               -
38963               -
38964               -
38965               -
38966               -
38967               -
38968               -
38969     

You can get multiple columns by specifying them in a list.

In [6]:
robocall_df[["phone_number", "type_telemarketing"]]

,phone_number,type_telemarketing
0,-,NaN
1,610-990-4243,Abandoned Calls
2,-,Prerecorded Voice
3,-,Abandoned Calls
4,-,NaN
5,-,Live Voice
6,-,Live Voice
7,-,Live Voice
8,-,Prerecorded Voice
9,-,Prerecorded Voice


And if we want to get just one row of that column we can use a second set of brackets with the row index ("13" in the example below).

In [7]:
robocall_df["phone_number"][13]

'321-608-4860'

In some cases you might want to change a piece of data, for instance in the process of cleaning it up. So any edits you make directly to the dataframe will be reflected in the data. Verify in the output below that row index 13 has had it's phone_number updated. In other cases you may want to replace many values at once which can be done using the `.replace()` [function](http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.DataFrame.replace.html)

In [8]:
robocall_df["phone_number"][13] = '404-608-4860'
robocall_df

,issues,time_issued,caller_id,phone_number,type_robo,type_telemarketing,state,date_issued,date_created
0,Telemarketing (including do not call and spoof...,NaN,-,-,NaN,NaN,Pennsylvania,NaN,2015-10-01
1,Telemarketing (including do not call and spoof...,2014-07-01 08:20:00,610-990-4243,610-990-4243,NaN,Abandoned Calls,Pennsylvania,2014-07-01,2015-10-01
2,Telemarketing (including do not call and spoof...,2015-09-20 14:00:00,469-656-8497,-,NaN,Prerecorded Voice,Pennsylvania,2015-09-20,2015-10-01
3,Telemarketing (including do not call and spoof...,2015-09-23 11:46:00,717-657-3334,-,NaN,Abandoned Calls,Pennsylvania,2015-09-23,2015-10-01
4,Robocalls,2015-09-28 15:19:00,415-946-5707,-,Prerecorded Voice,NaN,Pennsylvania,2015-09-28,2015-10-01
5,Telemarketing (including do not call and spoof...,2015-09-29 12:00:00,215-624-8359,-,NaN,Live Voice,Pennsylvania,2015-09-29,2015-10-01
6,Telemarketing (including do not call and spoof...,2015-10-01 11:00:00,202-470-3314,-,NaN,Live Voice,Pennsylvania,2015-10-01,2015-10-01
7,Telemarketing (including do not call and spoof...,2015-10-01 10:21:00,567-263-0009,-,NaN,Live Voice,Pennsylvania,2015-10-01,2015-10-01
8,Telemarketing (including do not call and spoof...,2015-10-01 13:02:00,215-763-3788,-,NaN,Prerecorded Voice,Pennsylvania,2015-10-01,2015-10-01
9,Telemarketing (including do not call and spoof...,2015-10-01 12:58:00,717-245-2434,-,NaN,Prerecorded Voice,Pennsylvania,2015-10-01,2015-10-01


### Filtering & Reindexing

Let's go back to our original data by reloading the file.

In [9]:
robocall_df = pd.read_csv("Data/Telemarketing_RoboCall_Weekly_Data_Transformed.csv")

You notice that a lot of datasets you work with are deficient in some way or another. For instance, they may be missing values in some rows and columns. When it loads a file in Pandas is smart enough to mark empty fields as "NaN" which stands for Not a Number. 

In [10]:
robocall_df["type_telemarketing"]

0                      NaN
1          Abandoned Calls
2        Prerecorded Voice
3          Abandoned Calls
4                      NaN
5               Live Voice
6               Live Voice
7               Live Voice
8        Prerecorded Voice
9        Prerecorded Voice
10                     NaN
11              Live Voice
12              Live Voice
13         Abandoned Calls
14              Live Voice
15              Live Voice
16                     NaN
17              Live Voice
18              Live Voice
19       Prerecorded Voice
20       Prerecorded Voice
21                     NaN
22                     NaN
23                     NaN
24       Prerecorded Voice
25       Prerecorded Voice
26              Live Voice
27       Prerecorded Voice
28       Prerecorded Voice
29              Live Voice
               ...        
38955      Abandoned Calls
38956                  NaN
38957                  NaN
38958           Live Voice
38959           Live Voice
38960                  NaN
3

We can test for these values using the ``isnull`` and ``notnull`` functions which will return a True / False value based on the value of the item. 

In [11]:
robocall_df["type_telemarketing"].isnull()

0         True
1        False
2        False
3        False
4         True
5        False
6        False
7        False
8        False
9        False
10        True
11       False
12       False
13       False
14       False
15       False
16        True
17       False
18       False
19       False
20       False
21        True
22        True
23        True
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
38955    False
38956     True
38957     True
38958    False
38959    False
38960     True
38961    False
38962    False
38963     True
38964     True
38965    False
38966    False
38967    False
38968    False
38969    False
38970    False
38971     True
38972    False
38973    False
38974    False
38975    False
38976    False
38977    False
38978    False
38979    False
38980     True
38981     True
38982     True
38983    False
38984    False
Name: type_telemarketing, dtype: bool

And we may want to filter out those empty values. We can do that with a special selector syntax. In the following notice that within the brackets we tell it to select rows for which type_telemarketing is not null. Another useful function for removing missing data is `dropna()` which has [parameters](http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.DataFrame.dropna.html) that allow you to drop rows or columns have have any or all values that are missing. 

In [12]:
robocall_df[robocall_df["type_telemarketing"].notnull()]

,issues,time_issued,caller_id,phone_number,type_robo,type_telemarketing,state,date_issued,date_created
1,Telemarketing (including do not call and spoof...,2014-07-01 08:20:00,610-990-4243,610-990-4243,NaN,Abandoned Calls,Pennsylvania,2014-07-01,2015-10-01
2,Telemarketing (including do not call and spoof...,2015-09-20 14:00:00,469-656-8497,-,NaN,Prerecorded Voice,Pennsylvania,2015-09-20,2015-10-01
3,Telemarketing (including do not call and spoof...,2015-09-23 11:46:00,717-657-3334,-,NaN,Abandoned Calls,Pennsylvania,2015-09-23,2015-10-01
5,Telemarketing (including do not call and spoof...,2015-09-29 12:00:00,215-624-8359,-,NaN,Live Voice,Pennsylvania,2015-09-29,2015-10-01
6,Telemarketing (including do not call and spoof...,2015-10-01 11:00:00,202-470-3314,-,NaN,Live Voice,Pennsylvania,2015-10-01,2015-10-01
7,Telemarketing (including do not call and spoof...,2015-10-01 10:21:00,567-263-0009,-,NaN,Live Voice,Pennsylvania,2015-10-01,2015-10-01
8,Telemarketing (including do not call and spoof...,2015-10-01 13:02:00,215-763-3788,-,NaN,Prerecorded Voice,Pennsylvania,2015-10-01,2015-10-01
9,Telemarketing (including do not call and spoof...,2015-10-01 12:58:00,717-245-2434,-,NaN,Prerecorded Voice,Pennsylvania,2015-10-01,2015-10-01
11,Telemarketing (including do not call and spoof...,2015-10-01 13:30:00,-,-,NaN,Live Voice,Pennsylvania,2015-10-01,2015-10-01
12,Telemarketing (including do not call and spoof...,2015-10-01 10:35:00,610-265-9391,-,NaN,Live Voice,Pennsylvania,2015-10-01,2015-10-01


If you need to filter by more than one column you can combine them using the ``&`` character. Though note you need an extra pair of parentheses around each logical test. Let's grab this dataframe and assign it to another variable. 

In [13]:
maryland_df = robocall_df[(robocall_df["type_telemarketing"].notnull()) & (robocall_df["state"]=="Maryland")]
maryland_df

,issues,time_issued,caller_id,phone_number,type_robo,type_telemarketing,state,date_issued,date_created
617,Telemarketing (including do not call and spoof...,2015-08-27 11:40:00,-,202-370-6665,NaN,Live Voice,Maryland,2015-08-27,2015-10-01
618,Telemarketing (including do not call and spoof...,2015-09-25 13:24:00,312-800-9056,-,NaN,Prerecorded Voice,Maryland,2015-09-25,2015-10-01
619,Telemarketing (including do not call and spoof...,2015-09-29 15:39:00,301-689-9705,-,NaN,Prerecorded Voice,Maryland,2015-09-29,2015-10-01
620,Telemarketing (including do not call and spoof...,2015-09-30 10:40:00,240-573-7979,-,NaN,Prerecorded Voice,Maryland,2015-09-30,2015-10-01
622,Telemarketing (including do not call and spoof...,2015-09-30 09:56:00,347-408-0225,347-408-0225,NaN,Live Voice,Maryland,2015-09-30,2015-10-01
623,Telemarketing (including do not call and spoof...,2015-09-30 17:01:00,703-260-6318,-,NaN,Prerecorded Voice,Maryland,2015-09-30,2015-10-01
624,Telemarketing (including do not call and spoof...,2015-10-01 15:00:00,786-452-4865,-,NaN,Prerecorded Voice,Maryland,2015-10-01,2015-10-01
625,Telemarketing (including do not call and spoof...,2015-10-01 14:14:00,657-202-9364,-,NaN,Prerecorded Voice,Maryland,2015-10-01,2015-10-01
626,Telemarketing (including do not call and spoof...,2015-10-01 17:30:00,443-910-0606,-,NaN,Prerecorded Voice,Maryland,2015-10-01,2015-10-01
629,Telemarketing (including do not call and spoof...,2015-10-01 09:50:00,202-795-2408,-,NaN,Prerecorded Voice,Maryland,2015-10-01,2015-10-01


After all that filtering you might wonder how much data you have left. To check the shape (i.e. number of rows and columns) of a DataFrame just append ``.shape`` at the end. 

In [14]:
maryland_df.shape

(774, 9)

You'll notice that in the filtered data frame the index starts from "617", but maybe we want to reset it to start at zero now that we're focused on Maryland. We can do that, but remember we have to assign the new dataframe back to the same name (i.e. `maryland_df`)

In [15]:
maryland_df = maryland_df.reset_index(drop=True)
maryland_df

,issues,time_issued,caller_id,phone_number,type_robo,type_telemarketing,state,date_issued,date_created
0,Telemarketing (including do not call and spoof...,2015-08-27 11:40:00,-,202-370-6665,NaN,Live Voice,Maryland,2015-08-27,2015-10-01
1,Telemarketing (including do not call and spoof...,2015-09-25 13:24:00,312-800-9056,-,NaN,Prerecorded Voice,Maryland,2015-09-25,2015-10-01
2,Telemarketing (including do not call and spoof...,2015-09-29 15:39:00,301-689-9705,-,NaN,Prerecorded Voice,Maryland,2015-09-29,2015-10-01
3,Telemarketing (including do not call and spoof...,2015-09-30 10:40:00,240-573-7979,-,NaN,Prerecorded Voice,Maryland,2015-09-30,2015-10-01
4,Telemarketing (including do not call and spoof...,2015-09-30 09:56:00,347-408-0225,347-408-0225,NaN,Live Voice,Maryland,2015-09-30,2015-10-01
5,Telemarketing (including do not call and spoof...,2015-09-30 17:01:00,703-260-6318,-,NaN,Prerecorded Voice,Maryland,2015-09-30,2015-10-01
6,Telemarketing (including do not call and spoof...,2015-10-01 15:00:00,786-452-4865,-,NaN,Prerecorded Voice,Maryland,2015-10-01,2015-10-01
7,Telemarketing (including do not call and spoof...,2015-10-01 14:14:00,657-202-9364,-,NaN,Prerecorded Voice,Maryland,2015-10-01,2015-10-01
8,Telemarketing (including do not call and spoof...,2015-10-01 17:30:00,443-910-0606,-,NaN,Prerecorded Voice,Maryland,2015-10-01,2015-10-01
9,Telemarketing (including do not call and spoof...,2015-10-01 09:50:00,202-795-2408,-,NaN,Prerecorded Voice,Maryland,2015-10-01,2015-10-01


### Accessing Rows

We may also sometimes need to access a row of data from a data frame. This can be done with the `iloc` accessor and providing the integer-based position in brackets, or with the `loc` accessesor and providing the label-based index in brackets. In this example the two are equivalent.

In [27]:
maryland_df.iloc[0]

issues                Telemarketing (including do not call and spoof...
time_issued                                         2015-08-27 11:40:00
caller_id                                                             -
phone_number                                               202-370-6665
type_robo                                                           NaN
type_telemarketing                                           live voice
state                                                          Maryland
date_issued                                                  2015-08-27
date_created                                                 2015-10-01
Name: 0, dtype: object

And if we need that row as an array we can use the `.values` which is sometimes necessary is we want to do other types of mathematical operations:

In [24]:
maryland_df.iloc[0].values

array(['Telemarketing (including do not call and spoofing)',
       '2015-08-27 11:40:00', '-', '202-370-6665', nan, 'live voice',
       'Maryland', '2015-08-27', '2015-10-01'], dtype=object)

### Applying Data Transformations

Sometimes you will want to transform your data by applying a transformation function to each datum within a column or row. We don't necessarily need to, but to show you how to do it, let's make all the text in the `type_telemarketing` column lowercase. We define a function which takes in an input datum (x in this case) and returns the transformed value of that. We use the `apply` [function](http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.DataFrame.apply.html) on the dataframe to apply that function to an entire column (or to an entire row).

In [18]:
def lowercaser(x):
    return x.lower()

maryland_df["type_telemarketing"] = maryland_df["type_telemarketing"].apply(lowercaser)

### Sorting 

Oftentimes you will want to sort your data to get an overview or see what is at the top or bottom of a ranking. To sort by values use the `sort_values` [function](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values). Below we sort by the time_issued column from most recent to least recent. 

In [19]:
maryland_df.sort_values(by="time_issued", ascending=False)

,issues,time_issued,caller_id,phone_number,type_robo,type_telemarketing,state,date_issued,date_created
771,Telemarketing (including do not call and spoof...,2015-12-19 21:00:00,240-720-3520,-,NaN,live voice,Maryland,2015-12-19,2015-12-19
772,Telemarketing (including do not call and spoof...,2015-12-19 18:51:00,703-348-5565,-,NaN,abandoned calls,Maryland,2015-12-19,2015-12-19
773,Telemarketing (including do not call and spoof...,2015-12-19 12:48:00,206-708-2849,-,NaN,prerecorded voice,Maryland,2015-12-19,2015-12-19
770,Telemarketing (including do not call and spoof...,2015-12-19 09:08:00,518-717-6542,-,NaN,live voice,Maryland,2015-12-19,2015-12-19
763,Telemarketing (including do not call and spoof...,2015-12-18 16:45:00,619-503-7074,-,NaN,prerecorded voice,Maryland,2015-12-18,2015-12-18
765,Telemarketing (including do not call and spoof...,2015-12-18 16:00:00,213-286-6011,213-286-6011,NaN,live voice,Maryland,2015-12-18,2015-12-18
769,Telemarketing (including do not call and spoof...,2015-12-18 15:25:00,-,-,NaN,abandoned calls,Maryland,2015-12-18,2015-12-18
764,Telemarketing (including do not call and spoof...,2015-12-18 14:50:00,-,-,NaN,live voice,Maryland,2015-12-18,2015-12-18
766,Telemarketing (including do not call and spoof...,2015-12-18 13:15:00,301-863-2426,-,NaN,abandoned calls,Maryland,2015-12-18,2015-12-18
767,Telemarketing (including do not call and spoof...,2015-12-18 13:07:00,818-474-2202,818-474-2202,NaN,live voice,Maryland,2015-12-18,2015-12-18


### Aggregation 

You'll often want to summarize DataFrames to get an overview of your data, or to aggregate it. The `describe()` function is useful for an initial overview, but there are many others such as `min()`, `max()`, `sum()`, `mean()`, and [many others](http://pandas.pydata.org/pandas-docs/stable/basics.html#descriptive-statistics)

In [20]:
maryland_df.describe()

,issues,time_issued,caller_id,phone_number,type_robo,type_telemarketing,state,date_issued,date_created
count,774,749,774,774,4,774,774,774,774
unique,1,737,494,138,2,5,1,95,76
top,Telemarketing (including do not call and spoof...,2015-11-10 12:50:00,-,-,Prerecorded Voice,live voice,Maryland,2015-11-16,2015-10-06
freq,774,2,166,634,3,295,774,22,22


A useful analytic operation is to create groups that can then be summarized. This can be accomplished with the `groupby()` [function](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html). Various [aggregation functions](http://pandas.pydata.org/pandas-docs/stable/groupby.html) can then be applied. 

In [21]:
state_groups = robocall_df.groupby("state")
state_groups.count()

,issues,time_issued,caller_id,phone_number,type_robo,type_telemarketing,date_issued,date_created
state,,,,,,,,
Alabama,505,481,505,505,134,366,504,505
Alaska,40,38,40,40,10,30,40,40
American Samoa,1,1,1,1,1,0,1,1
Arizona,890,850,890,890,334,550,884,890
Arkansas,245,237,245,245,69,174,243,245
California,5246,5019,5246,5246,1719,3517,5220,5246
Colorado,1030,985,1030,1030,359,669,1028,1030
Connecticut,672,645,672,672,234,437,669,672
Delaware,141,133,141,141,34,106,141,141


### Deduplication

At times your data will have duplicate rows in it and you'll want to remove those. To check for duplicated rows you can use the `.duplicated()` [function](http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.DataFrame.duplicated.html), and if you want to check for duplicates within a certain column you can pass that as a parameter. Let's say we want to detect duplicate caller id numbers: 

In [22]:
maryland_df.duplicated(["caller_id"])

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10      True
11     False
12     False
13     False
14     False
15     False
16     False
17      True
18     False
19     False
20     False
21     False
22      True
23     False
24      True
25     False
26     False
27     False
28      True
29     False
       ...  
744     True
745     True
746    False
747    False
748    False
749    False
750    False
751    False
752    False
753     True
754    False
755    False
756     True
757    False
758    False
759    False
760     True
761    False
762     True
763    False
764     True
765    False
766    False
767    False
768     True
769     True
770    False
771    False
772    False
773     True
dtype: bool

We can then drop the rows detected as duplicates using the `drop_duplicates()` [function](http://pandas.pydata.org/pandas-docs/version/0.17.1/generated/pandas.DataFrame.drop_duplicates.html). 

In [23]:
maryland_df.drop_duplicates(["caller_id"]).shape

(494, 9)